In [1]:
import glob

import numpy as np
import pandas as pd

from grafting_classifier import GraftingClassifier
from sklearn.linear_model import SGDClassifier
from ogfs_classifier import OGFSClassifier
from osfs_classifier import OSFSClassifier
from dpp_classifier import DPPClassifier
from dpp_classifier_mitra import DPPClassifier as DPPClassifier2
from dpp_classifier_ogfs import DPPClassifier as DPPClassifier3

from sklearn.metrics import log_loss, accuracy_score

#import dask.dataframe as dd
#import dask.array as da

In [2]:
class_train = glob.glob("uci/*_train.csv")
print(class_train)

['uci\\Ionosphere_train.csv', 'uci\\spambase_train.csv', 'uci\\spectf_train.csv', 'uci\\wdbc_train.csv']


In [3]:
def train_label(fname):
    targetname = fname.replace(".csv", ".labels")
    return pd.read_csv(targetname)

In [4]:
def get_performance(mod, fpath, base=False):
    train1 = pd.read_csv(fpath).fillna(0)
    y = np.array(train_label(fpath)).flatten()
    
    # simulate streaming...
    # try splitting into groups of ~10,
    # if there is no splits, try ~5.
    train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/10.0) + 1)
    if len(train1_cols) == 1:
        train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/5.0) + 1)
    all_cols = []

    #mod = GraftingClassifier(max_iter=5)
    if base:
        mod.fit(train1, y)
        results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
        return results
    
    # lets normalise the dataset...
    train1 = (train1 - train1.mean())/(np.maximum(train1.std(), 1))
    for idx, collist in enumerate(train1_cols):
        if idx == 0:
            column_list = list(np.array(list(train1.columns))[collist])
            mod.fit(train1[column_list], y)
            all_cols.extend(list(collist))
        else:
            all_cols.extend(list(collist))
            column_list = list(np.array(list(train1.columns))[all_cols])
            mod.partial_fit(train1[column_list], y)
    
    results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
    return results

In [5]:
def create_models():
    return [
    ('Grafting', GraftingClassifier(max_iter=5, random_state=42)), 
    ('DPP', DPPClassifier(max_iter=5, random_state=42)), 
    #('DPP2', DPPClassifier2(max_iter=5, random_state=42)),
    ('DPP3', DPPClassifier3(max_iter=5, random_state=42)),
    ('OGFS', OGFSClassifier(max_iter=5, random_state=42)),
    ('OSFS', OSFSClassifier(max_iter=5, random_state=42)),
    ('Base', SGDClassifier(loss='log', max_iter=5, random_state=42))
]

In [6]:
ex_dat = class_train[0]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

uci\Ionosphere_train.csv (351, 34)
Grafting {'accuracy': 0.90028490028490027, 'logloss': 0.74098002095622384, 'feat_dim': (32,)}
DPP {'accuracy': 0.86894586894586889, 'logloss': 0.48697468342748756, 'feat_dim': (9,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:79: RuntimeWarning: divide by zero encountered in true_divide
  X = s_b/s_w
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:98: RuntimeWarning: divide by zero encountered in double_scalars
  prev_score = np.sum(curr_u1)/np.sum(curr_u2)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:93: RuntimeWarning: divide by zero encountered in double_scalars
  score = ((np.sum(test_u1)/np.sum(test_u2)) - prev_score)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:93: RuntimeWarning: invalid value encountered in double_scalars
  score = ((np.sum(test_u1)/np.sum(test_u2)) - prev_score)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.py:143: RuntimeWarning: divide by zero encountered in true_divide
  eval2 = s_b/s_w
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\dpp_classifier_ogfs.p

DPP3 {'accuracy': 0.86039886039886038, 'logloss': 0.74610861608362322, 'feat_dim': (23,)}
OGFS {'accuracy': 0.87749287749287752, 'logloss': 0.66038083699105898, 'feat_dim': (26,)}
OSFS {'accuracy': 0.79487179487179482, 'logloss': 1.1849841892721311, 'feat_dim': (5,)}
Base {'accuracy': 0.91737891737891741, 'logloss': 1.9154349277794036, 'feat_dim': (34,)}


In [7]:
ex_dat = class_train[1]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

uci\spambase_train.csv (4601, 57)
Grafting {'accuracy': 0.92088676374701151, 'logloss': 0.26983268043411535, 'feat_dim': (50,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\kernel_approximation.py:470: UserWarning: n_components > n_samples. This is not possible.
n_components was set to n_samples, which results in inefficient evaluation of the full kernel.
  warnings.warn("n_components > n_samples. This is not possible.\n"
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_in

DPP {'accuracy': 0.81938708976309493, 'logloss': 0.46031455107612113, 'feat_dim': (10,)}
DPP3 {'accuracy': 0.88024342534231692, 'logloss': 0.35987306878143693, 'feat_dim': (37,)}
OGFS {'accuracy': 0.91241034557704848, 'logloss': 0.29388171935398261, 'feat_dim': (24,)}
OSFS {'accuracy': 0.81156270376005213, 'logloss': 0.49108155193168401, 'feat_dim': (7,)}
Base {'accuracy': 0.708324277331015, 'logloss': 10.070713136799997, 'feat_dim': (57,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [8]:
ex_dat = class_train[2]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

uci\spectf_train.csv (267, 44)
Grafting {'accuracy': 0.797752808988764, 'logloss': 0.5767922374772857, 'feat_dim': (37,)}
DPP {'accuracy': 0.79400749063670417, 'logloss': 0.63745048472449595, 'feat_dim': (31,)}
DPP3 {'accuracy': 0.79400749063670417, 'logloss': 0.70656341135636747, 'feat_dim': (29,)}


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\SPEC.py:114: RuntimeWarning: invalid value encountered in power
  d1 = np.power(np.array(W.sum(axis=1)), -0.5)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming\SPEC.py:116: RuntimeWarning: invalid value encountered in power
  d2 = np.power(np.array(W.sum(axis=1)), 0.5)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


OGFS {'accuracy': 0.79400749063670417, 'logloss': 0.74690754658996694, 'feat_dim': (5,)}
OSFS {'accuracy': 0.79400749063670417, 'logloss': 0.78399720192355737, 'feat_dim': (16,)}
Base {'accuracy': 0.79400749063670417, 'logloss': 7.1147292199254215, 'feat_dim': (44,)}


In [9]:
ex_dat = class_train[3]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

uci\wdbc_train.csv (569, 30)
Grafting {'accuracy': 0.94200351493848855, 'logloss': 0.19962258813204223, 'feat_dim': (24,)}
DPP {'accuracy': 0.86291739894551844, 'logloss': 0.38867023987714688, 'feat_dim': (13,)}
DPP3 {'accuracy': 0.86643233743409487, 'logloss': 0.38675676693202549, 'feat_dim': (13,)}
OGFS {'accuracy': 0.95782073813708257, 'logloss': 0.10190076800196465, 'feat_dim': (18,)}
OSFS {'accuracy': 0.77328646748681895, 'logloss': 0.46301339093328314, 'feat_dim': (10,)}
Base {'accuracy': 0.91564147627416526, 'logloss': 2.9136401879713767, 'feat_dim': (30,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
